In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#paste your sale report file path here
df = pd.read_csv(r"file_path")

In [ ]:
df.rename(columns={"Display Order Code":"order_id","Item SKU Code":"sku"},inplace=True)
df=df[df['Shipping Package Creation Date']=="CREATED"]
#df['sku']=df['sku'].astype('category')
#label1 = dict(enumerate(df['sku'].cat.categories))
df['sku']=df['sku'].astype('category').cat.codes
df.sort_values(by=["order_id","sku"],inplace=True)

In [ ]:
#combo data vs frequency
df_a=df[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df_a['combo']=df[['order_id','sku']].groupby('order_id')['sku'].apply(list).reset_index(name='combo')['combo']
df_a['combo_dist_tuple']=df[['order_id','sku']].groupby('order_id')['sku'].unique().apply(tuple).reset_index(name='combo_dt')['combo_dt']
tuple_qty=[]
for i in df_a['combo_dist_tuple']:
    len_tuple=len(i)
    tuple_qty.append(len_tuple)
df_a['len_tuple']=tuple_qty   

In [ ]:
df_a.head()

,order_id,qty,combo,combo_dist_tuple,len_tuple
0,91012506548-1,2,"[147, 219]","(147, 219)",2
1,91012509181-1,1,[167],"(167,)",1
2,91012514268-1,1,[66],"(66,)",1
3,91012517429-1,2,"[12, 298]","(12, 298)",2
4,91012520756-1,1,[31],"(31,)",1


In [ ]:
df_b=df_a.groupby('combo_dist_tuple').size().reset_index(name='freq').sort_values('freq',ascending=False)
df_b['order_list']=df_a.groupby('combo_dist_tuple')['order_id'].apply(list).reset_index(name='a')['a']
df_b.head(10)

,combo_dist_tuple,freq,order_list
368,"(3, 74)",238,"[91012676036, 91012677773, 91012678783, 910126..."
257,"(3,)",149,"[91012675908, 91012676311, 91012676352, 910126..."
234,"(2, 63)",109,"[91012658557, 91012669269, 91012681760, 910126..."
681,"(8,)",105,"[91012675095, 91012675282, 91012675576, 910126..."
2030,"(34,)",104,"[91012675031, 91012675162, 91012675374, 910126..."
2453,"(55,)",103,"[91012673640, 91012675369, 91012676617, 910126..."
3098,"(146,)",86,"[91012657798, 91012658124, 91012673318, 910126..."
1822,"(27,)",67,"[91012675595, 91012676046, 91012676296, 910126..."
157,"(2,)",58,"[91012674150, 91012675303, 91012675343, 910126..."
360,"(3, 58, 74)",56,"[91012669185, 91012670629, 91012673796, 910126..."


In [ ]:
min_freq=25 ##select min frequency which you want
result_combo=df_b[df_b['freq']>=min_freq]  

In [ ]:
result_combo

,combo_dist_tuple,freq,order_list
368,"(3, 74)",238,"[91012676036, 91012677773, 91012678783, 910126..."
257,"(3,)",149,"[91012675908, 91012676311, 91012676352, 910126..."
234,"(2, 63)",109,"[91012658557, 91012669269, 91012681760, 910126..."
681,"(8,)",105,"[91012675095, 91012675282, 91012675576, 910126..."
2030,"(34,)",104,"[91012675031, 91012675162, 91012675374, 910126..."
2453,"(55,)",103,"[91012673640, 91012675369, 91012676617, 910126..."
3098,"(146,)",86,"[91012657798, 91012658124, 91012673318, 910126..."
1822,"(27,)",67,"[91012675595, 91012676046, 91012676296, 910126..."
157,"(2,)",58,"[91012674150, 91012675303, 91012675343, 910126..."
360,"(3, 58, 74)",56,"[91012669185, 91012670629, 91012673796, 910126..."


In [ ]:
result_combo.freq.sum()

1733

In [ ]:
result1=pd.DataFrame(columns=['OrderID','Tracking','code','batch','forceupdate','Channel_code'])
k=0
l=1
for i in result_combo['order_list']:
  for j in i:
    result1.at[k,'OrderID']=j
    result1.at[k,'Channel_code']='Mamaearth_Production'
    result1.at[k,'batch']=l
    k=k+1
  l=l+1
   
result1.to_excel(r"combo_result.xlsx")

In [ ]:
#now finding max order containing SKU pool

#preparing rest of the data after removing combo orders ids

df_rest=df_b[df_b['freq']<min_freq]

rest_orders=[]
for i in df_rest.order_list:
    rest_orders.extend(i)


df_rest=df[df['order_id'].isin(rest_orders)]
df_a=df_a[df_a['order_id'].isin(rest_orders)]

In [ ]:
"""
    min_freq=a
    combo_size=b
    n=c
    df_rest=d.copy()
    df_a=e.copy()
"""

In [ ]:
 ##combo size

################################ Combo formation ###################################

def combo_set(min_freq,combo_size,n,df_rest,df_a):

    df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
    topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
    topn.sort()

    if combo_size==3:    
        a=[]
        combo=[None]*combo_size
        for i in range(len(topn)):
            for j in range(i+1,len(topn)):
                for k in range(j+1,len(topn)):
                    combo[0]=topn[i]
                    combo[1]=topn[j]
                    combo[2]=topn[k]
                    a.append(combo.copy())

    elif combo_size==4:    
        a=[]
        combo=[None]*combo_size
        for i in range(len(topn)):
            for j in range(i+1,len(topn)):
                for k in range(j+1,len(topn)):
                    for l in range(k+1,len(topn)):
                        combo[0]=topn[i]
                        combo[1]=topn[j]
                        combo[2]=topn[k]
                        combo[3]=topn[l]
                        a.append(combo.copy())
                   
    elif combo_size==5:
        a=[]
        combo=[None]*combo_size
        for i in range(len(topn)):
            for j in range(i+1,len(topn)):
                for k in range(j+1,len(topn)):
                    for l in range(k+1,len(topn)):
                        for m in range(l+1,len(topn)):
                            combo[0]=topn[i]
                            combo[1]=topn[j]
                            combo[2]=topn[k]
                            combo[3]=topn[l]
                            combo[4]=topn[m]
                            a.append(combo.copy())
    result_df=pd.DataFrame(columns=['combo',"freq"])
    result_df['combo']=a

    
    
    df3=df_rest[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
    df3['sku_list']=df_rest[['order_id','sku']].groupby('order_id')['sku'].unique().apply(list).reset_index(name='combo')['combo']
    
    #len(a)
    ## Searching number of fully inclusive orders in a given pair of SKUS

    length_orders=df3['order_id'].count()

    res_list=[]
    for sample in result_df['combo']:
        mfg=0
        for j in range(length_orders):
            test_list=df3.sku_list[j]
            check=all(item in sample for item in test_list) 
            if check is True:
                mfg=mfg+1
            else:
               mfg=mfg
        res_list.append(mfg)


    ##############################################    
    result_df['freq']=res_list
    result_df=result_df.sort_values("freq",ascending=False)
    result_df=result_df[result_df['freq']>min_freq]
    result_df.reset_index(drop=True,inplace=True)

    distinct=[]
    distinct.append(list(result_df['combo'][0]))
    list_a=result_df['combo'][0].copy()
    for k in range(len(result_df['combo'])):
      list_b=result_df['combo'][k]
      c = [value for value in list_a if value in list_b]
      if len(c)==0:
        distinct.append(list_b)
        list_a.extend(list_b)
    result_df=result_df[result_df['combo'].isin(distinct)]
    #######################################################
            
    result_df = result_df.reset_index(drop=True)
    include_list=[]
    combo_rest=result_df.copy()
    combo_rest['order_list']=''

    for i in result_df.index:
        combo=result_df['combo'][i]
        df_x=df_a[['order_id','combo']]
        abc=[]
        for sample in df_x.combo:
            test=all(item in combo for item in sample)
            if test is True:
                abc.append('yes')
            else:
                abc.append('no')
        df_x['order_in_pool']=abc
        p=df_x[df_x['order_in_pool']=='yes']['order_id'].to_list()
        s=p.copy()
        combo_rest.at[i,'order_list']=s
        for i in combo_rest['order_list']:
          for j in i:
            include_list.append(j)

    df_rest=df_rest[df_rest['order_id'].isin(include_list)==False]
    df_a=df_a[df_a['order_id'].isin(include_list)==False]

    return df_rest,df_a,combo_rest

In [ ]:
df_rest1,df_a1,combo_rest1=combo_set(20,3,15,df_rest,df_a)

<ipython-input-218-0e026bfa8d03>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=abc


In [ ]:
combo_rest1

,combo,freq,order_list
0,"[4, 15, 57]",60,"[91012658737, 91012658974, 91012660547, 910126..."
1,"[3, 34, 74]",53,"[91012657921, 91012667430, 91012668002, 910126..."


In [ ]:
df_rest1,df_a1,combo_rest1=combo_set(20,3,15,df_rest1,df_a1)

<ipython-input-218-0e026bfa8d03>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=abc


In [ ]:
combo_rest1

,combo,freq,order_list
0,"[36, 81, 98]",37,"[91012657926, 91012668351, 91012669113, 910126..."


In [ ]:
result=pd.DataFrame(columns=['OrderID','Tracking','code','batch','forceupdate','Channel_code'])
k=0
l=1
for i in combo_rest1['order_list']:
  for j in i:
    result.at[k,'OrderID']=j
    result.at[k,'Channel_code']='Mamaearth_Production'
    result.at[k,'batch']=l
    k=k+1
  l=l+1
   
result.to_excel(r"result1.xlsx")